# Daily data download and first processing

This notebook is part of a toolset to analyse and visualise data on the COVID-19 epidimic and will be run once a day.<br>
Though this can only be done locally on my machine, the data will be saved in ../data/ in this repository.

We will download the ECDC's daily datasheet (https://ourworldindata.org/coronavirus-source-data), containing amount of new cases and deaths due to the COVID-19 outbreak per country per day as an excel sheet.<br>
After which we will slip it into three datasets: daily cases per country, daily deaths per country, and a set containing the populations of the countries in 2018.

I will comment allong the way on what is happening and regularly print what we're working with for transparency.

## Downloading

First we'll have to download the ECDC datasheet.

In [1]:
import urllib.request               # Calls url to downlload daily file
import datetime                     # Provides current date

try:
    today = datetime.date.today() - datetime.timedelta(days=1)
    url = 'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-' + str(today) + '.xlsx'

    # Second argument in urlretrieve is the destination and filename for saving
    _ = urllib.request.urlretrieve(url, '../data/ECDC_data.xlsx')
    
except: 
    print("No new data.")

Next we'll need to open the downloaded .xlsx file as a pandas dataframe so we can easelly manipulate the data.
Pandas is a powerfull python library to create and manipulate dataframes that is used throughout this project.

We won't go into detail how to use this library, since that would be to expansive for this project. Though it is very 'googleable' and the documenation is expansive.

In [2]:
import pandas as pd                   # Tool to manipulate datasets

df = pd.read_excel('../data/ECDC_data.xlsx')
df

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,2020-04-01,1,4,2020,25,0,Afghanistan,AF,AFG,37172386.0
1,2020-03-31,31,3,2020,27,0,Afghanistan,AF,AFG,37172386.0
2,2020-03-30,30,3,2020,8,1,Afghanistan,AF,AFG,37172386.0
3,2020-03-29,29,3,2020,15,1,Afghanistan,AF,AFG,37172386.0
4,2020-03-28,28,3,2020,16,1,Afghanistan,AF,AFG,37172386.0
...,...,...,...,...,...,...,...,...,...,...
8097,2020-03-25,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
8098,2020-03-24,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0
8099,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
8100,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0


## Processing

Now that we have the data in a form in which we can mold it, we can start working.<br>
As a first step we will clean out the table a bit since we have quite some values that we do not need.

Considering the 'DateRep' column contains a datetime that Python can translate we will delete the unneeded time attributes, as well as the geoid's since I will not be needing them yet.
This can be changed at a later time in case we want to combine other datasets containing country data.

In [3]:
df = df.drop(columns=['day', 'month', 'year', 'geoId', 'countryterritoryCode'])
display(df)

,dateRep,cases,deaths,countriesAndTerritories,popData2018
0,2020-04-01,25,0,Afghanistan,37172386.0
1,2020-03-31,27,0,Afghanistan,37172386.0
2,2020-03-30,8,1,Afghanistan,37172386.0
3,2020-03-29,15,1,Afghanistan,37172386.0
4,2020-03-28,16,1,Afghanistan,37172386.0
...,...,...,...,...,...
8097,2020-03-25,0,0,Zimbabwe,14439018.0
8098,2020-03-24,0,1,Zimbabwe,14439018.0
8099,2020-03-23,0,0,Zimbabwe,14439018.0
8100,2020-03-22,1,0,Zimbabwe,14439018.0


To make our next step a little easier we will create two lists. One containing every date and another containing every country that can be found in the database. Both will be ordered so we can fill in all the datasets in the same alphabetical/accending order.<br>
We will also exchange the _'s in the country name for a whitespace, since this looks better.

In [4]:
dates = []
countries = []

for date in df['dateRep']:
    if date not in dates:
        dates.append(date)

for country in df['countriesAndTerritories']:
    if country not in countries:
        countries.append(country)
        
dates.sort()
countries.sort()

countries_final = [country.replace('_', ' ') for country in countries]

Now for the actual data extraction. 

When we create datasets with new dataframe (aka new column or row structure), we will do this by first creating a dictionary. This will allow us to, in a structured way, collect and order the data by making lists, which we can then fold into dataframe.<br>
This dictionaries will be filled with the names of the columns, coupled to the data of that column in list form.

We will use the dates as an index, going from the earliest date to the latest. This means for each country we will need to create two list's, in that same order, containing cases and deaths per day. To be able to add these lists, with country name as key, into their respective dictionaries.

In [5]:
dict_cases = {'Date': dates}
dict_deaths = {'Date': dates}

# here we need a numerical for loop, instead of loping through the countries list
# since we need to place the countries_final name in the dictionary instead of the countries one
for i in range(len(countries)):
    country_pop = df.loc[df['countriesAndTerritories']==countries[i]]['popData2018'].values[-1]
    
    cases = []
    deaths = []
    
    country_df = df.loc[df['countriesAndTerritories']==countries[i]]
    collected_dates = country_df['dateRep'].tolist()
    
    for date in dates:
        if date in collected_dates:
            entry = country_df.loc[country_df['dateRep']==date]
            cases.append(entry['cases'].values[0])
            deaths.append(entry['deaths'].values[0])
            
        else:
            cases.append(0)
            deaths.append(0)
    
    dict_cases[countries_final[i]] = cases
    dict_deaths[countries_final[i]] = deaths

New we're finally ready to create the dataframes for both daily cases and daily deaths. 

We will start with the daily cases. While creating this, for easy visualization, we will place the date as index and fill all the empty cells, when there was no data provided, with 0.

In [6]:
df_cases_daily = pd.DataFrame(dict_cases).set_index('Date')
df_cases_daily = df_cases_daily.fillna(0)

display(df_cases_daily)

,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,...,United Kingdom,United Republic of Tanzania,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-28,16,12,0,43,1,0,0,101,43,0,...,2885,0,2,18695,0,21,12,16,2,2
2020-03-29,15,11,104,41,0,0,0,55,52,0,...,2546,0,3,19979,66,29,0,54,12,2
2020-03-30,8,15,45,26,3,0,0,75,0,22,...,2433,1,8,18360,5,12,0,5,1,0


For the daily deaths we will do the same thing.

In [7]:
df_deaths_daily = pd.DataFrame(dict_deaths).set_index('Date')
df_deaths_daily = df_deaths_daily.fillna(0)

display(df_deaths_daily)

,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,...,United Kingdom,United Republic of Tanzania,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-28,1,3,0,0,0,0,0,5,0,0,...,181,0,0,411,0,0,0,0,0,0
2020-03-29,1,1,5,1,0,0,0,2,2,0,...,260,1,0,484,0,1,0,0,0,0
2020-03-30,1,0,3,2,2,0,0,1,0,0,...,209,0,0,318,1,0,0,0,0,0


With that done, the only thing left to do is to get the populations out of ECDC datasheet. Originally this was done at the same time as the previous data extraction, since it uses the same loop though it seemed more transparent to split it up.

Again we will make a dictionary first. Though this time we will make the countries the index, saving us the trouble of having to deal with the countries_final name in the loop itself.

In [8]:
dict_populations = {'Countries': countries_final}

populations = []
for country in countries:
    country_pop = df.loc[df['countriesAndTerritories']==country]['popData2018'].values[-1]
    populations.append(country_pop)
    
dict_populations['populations'] = populations

Again we will use this dictionary to create the dataframe.

In [9]:
df_populations = pd.DataFrame(dict_populations).set_index('Countries')
df_populations = df_populations.fillna(0)

display(df_populations)

,populations
Countries,
Afghanistan,37172386.0
Albania,2866376.0
Algeria,42228429.0
Andorra,77006.0
Angola,30809762.0
...,...
Uzbekistan,32955400.0
Venezuela,28870195.0
Vietnam,95540395.0


## Saving

With all the datasets created we can now save them to csv files in ../data/, to be used by the other notebooks for further processing, visualisations and later on to train networks and regression models.

It will always save over the same file so that all other applications can just rerun when the data is updated and the data is available in the ../data/ directory.

In [10]:
df_cases_daily.to_csv('../data/cases_daily.csv')
df_deaths_daily.to_csv('../data/deaths_daily.csv')
df_populations.to_csv('../data/populations_2018.csv')